## Homework 10

1. 
Inductive Reasoning: Taking steps to determine a known solution. Through inductive reasoning one combines pieces of information that might seem unrelated in hopes of forming general relationships. In doing so, one often notices patterns or connections. An example of this would be if a student found every one of his Taming Big Data assignments to be difficult, it is likely that the next one will also be strenous. To add to this, just because we draw this general conclusion, it doesn't necessarily mean our determination will be correct. 

Deductive Reasoning: Using a set of factual observations deem something as true. For example if every Cardinal player is over 22, you could state that Nolan Arenado is over 22. Of course, it can become much more complicated than this, but the purpose is that one deduces factors of the hypothesis to determine if it is indeed true. 

Sources:
https://7esl.com/inductive-vs-deductive-reasoning/

https://www.uis.edu/ion/resources/oiai/inductive-reasoning


In [1]:


import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost import XGBClassifier

/var/folders/kq/22rtshv9635518wrmfbx3l1r0000gn/T/ipykernel_46816/2632663677.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# 2. 

starting_data_df = pd.read_csv("Credit_Card.csv")
starting_data_df.head()


,Ind_ID,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,Housing_type,Birthday_count,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Type_Occupation,Family_Members
0,5008827,M,Y,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-18772.0,365243,1,0,0,0,NaN,2
1,5009744,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2
2,5009746,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,NaN,-586,1,1,1,0,NaN,2
3,5009749,F,Y,N,0,NaN,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2
4,5009752,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2


In [3]:
data_label_df = pd.read_csv("Credit_card_label.csv")

In [4]:
data_df = pd.merge(starting_data_df, data_label_df, on="Ind_ID")
data_df.head()

,Ind_ID,GENDER,Car_Owner,Propert_Owner,CHILDREN,Annual_income,Type_Income,EDUCATION,Marital_status,Housing_type,Birthday_count,Employed_days,Mobile_phone,Work_Phone,Phone,EMAIL_ID,Type_Occupation,Family_Members,label
0,5008827,M,Y,Y,0,180000.0,Pensioner,Higher education,Married,House / apartment,-18772.0,365243,1,0,0,0,NaN,2,1
1,5009744,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1
2,5009746,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,NaN,-586,1,1,1,0,NaN,2,1
3,5009749,F,Y,N,0,NaN,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1
4,5009752,F,Y,N,0,315000.0,Commercial associate,Higher education,Married,House / apartment,-13557.0,-586,1,1,1,0,NaN,2,1


In [5]:
data_df.fillna(method="ffill", inplace=True)


/var/folders/kq/22rtshv9635518wrmfbx3l1r0000gn/T/ipykernel_46816/3742191670.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_df.fillna(method="ffill", inplace=True)


In [6]:
print(data_df.columns)

Index(['Ind_ID', 'GENDER', 'Car_Owner', 'Propert_Owner', 'CHILDREN',
       'Annual_income', 'Type_Income', 'EDUCATION', 'Marital_status',
       'Housing_type', 'Birthday_count', 'Employed_days', 'Mobile_phone',
       'Work_Phone', 'Phone', 'EMAIL_ID', 'Type_Occupation', 'Family_Members',
       'label'],
      dtype='object')


In [7]:
column_titles = ['Ind_ID', 'Gender', 'Car_owner', 'Propert_owner', 'Children', 'Annual_income', 
                'Type_Income', 'Education', 'Marital_status', 'Housing_type', 'Birthday_count', 
                'Employed_days', 'Mobile_phone', 'Work_phone', 'Phone', 'EMAIL_ID', 
                'Type_Occupation', 'Family_Members','label']
data_df.columns = column_titles

In [8]:
data_df.drop(columns=['Ind_ID','Birthday_count','Mobile_phone','Work_phone','EMAIL_ID'], inplace=True)

In [9]:
scaler = StandardScaler()
numerical_columns = ['Children', 'Annual_income', 'Employed_days', 'Family_Members']
data_df[numerical_columns] = scaler.fit_transform(data_df[numerical_columns])

In [10]:
X = data_df.drop('label', axis=1)  
y = data_df['label']

In [11]:
X_dummy = pd.get_dummies(X)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_dummy,y,
                                                    test_size=0.3, random_state=32)

In [13]:
#3. 
model = tree.DecisionTreeClassifier(max_depth = 7, random_state=32)

In [14]:
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94       416
           1       0.46      0.12      0.19        49

    accuracy                           0.89       465
   macro avg       0.68      0.55      0.57       465
weighted avg       0.86      0.89      0.86       465



In [16]:
# To tune this model I played around with the max depth of the tree to help me come 
# to the best weighted average precision I could achieve. The more depth, the more complex
# the model. A model that has too much depth can cause overfitting. 

In [23]:
# 4. 
rf = RandomForestClassifier(n_estimators=60, random_state=32)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)



0.9290322580645162

In [ ]:
# The key factor with the Random Forest Model is the number of estimators. 
# Typically, increasing the number of estimators or trees will aid in making more accurate predictions
# However, at a certain point computational costs will be affected for what may only make 
# an extremely marginal difference.
# In order to demostrate that there was not much of a jump in the results I was getting upon
# increasing the estimators from 60, I chose this number as a good example to show that the 
# amount of trees does not have to be large. 

In [34]:
# 5.
model = xgb.XGBClassifier(random_state=32, n_estimators=100, max_depth=50, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       416
           1       0.74      0.47      0.57        49

    accuracy                           0.93       465
   macro avg       0.84      0.73      0.77       465
weighted avg       0.92      0.93      0.92       465



In [ ]:
# With XGBoost, I played with the max depth to raise the averages as much as I could
# particuraly focusing on precision, which makes the most sense for evaluating a model 
# concerned with credit approval as opposed to medical datasets as I have used in the past. 